# **Descargar datos**

In [2]:
import pandas as pd
import requests
import time
import os
from datetime import datetime

In [2]:
def get_crypto_prices(crypto_id, currency="usd", days=365):

    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {"vs_currency": currency, "days": days, "interval": "daily"}
    response = requests.get(url, params=params)

    # Si la respuesta es exitosa, se procesan los datos
    if response.status_code == 200:

        # Se convierte la respuesta a un DataFrame
        data = response.json()["prices"]
        df = pd.DataFrame(data, columns=["timestamp", "adjusted_price"])
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

        # Se guarda el archivo en la carpeta "crypto_data_YYYY-MM-DD"
        today = datetime.now().strftime("%Y-%m-%d")
        folder = f"crypto_data_{today}_{days}"
        if not os.path.exists(folder):
            os.makedirs(folder)

        filename = f"{folder}/{crypto_id}_prices.csv"
        df.to_csv(filename, index=False)

        print(f"Precios de {crypto_id} guardados en {filename}")        
        return None # No hay errores
    
    
    # Si la respuesta no es exitosa, se imprime un mensaje de error
    else:
        print(f"Error al obtener datos de {crypto_id}: {response.status_code}")
        
        if response.status_code == 429:
            print(f"Demasiadas peticiones. Esperando {response.headers['Retry-After']} segundos.")
            time.sleep(int(response.headers["Retry-After"]))
            get_crypto_prices(crypto_id, currency, days)
            
        return response.status_code

In [3]:
# Lista de 50 criptomonedas
cryptos = [
    "bitcoin", "ethereum", "tether", "binancecoin", "usd-coin",
    "ripple", "cardano", "solana", "dogecoin", "polkadot",
    "shiba-inu", "dai", "tron", "avalanche-2", "wrapped-bitcoin",
    "uniswap", "litecoin", "chainlink", "cosmos", "stellar",
    "monero", "algorand", "bitcoin-cash", "vechain", "filecoin",
    "internet-computer", "hedera", "decentraland", "the-sandbox", "tezos",
    "elrond-erd-2", "theta-token", "aave", "eos", "pancakeswap-token",
    "flow", "zcash", "maker", "klay-token", "neo",
    "iota", "fantom", "kusama", "waves", "dash",
    "synthetix-network-token", "compound", "chiliz", "enjincoin"
]

In [ ]:
for crypto in cryptos:
    get_crypto_prices(crypto, days=365)

Guardar todos los datos en uno

In [3]:
csv_folder = "crypto_data_2024-12-19_365"  # Carpeta donde están los CSV

In [4]:
all_data = {}  # Diccionario para almacenar datos de cada criptomoneda

for file in os.listdir(csv_folder):
    if file.endswith(".csv"):
        crypto_name = file.replace("_prices.csv", "")  # Obtener nombre de la cripto
        file_path = os.path.join(csv_folder, file)
        df = pd.read_csv(file_path)  # Leer el CSV
        df["timestamp"] = pd.to_datetime(df["timestamp"])  # Asegurar formato datetime
        df = df.set_index("timestamp")  # Usar fecha como índice
        all_data[crypto_name] = df["adjusted_price"]  # Guardar precios ajustados

# Crear DataFrame con todos los datos
data_crypto = pd.DataFrame(all_data)

In [7]:
data_crypto.dropna(inplace=True)  # Eliminar filas con valores nulos

In [8]:
data_crypto.to_csv("data_crypto.csv")

In [9]:
print(data_crypto.head())

                  aave  algorand  avalanche-2  binancecoin  bitcoin-cash  \
timestamp                                                                  
2023-12-22  101.455626  0.223064    46.054342   271.073933    233.085976   
2023-12-23  100.128661  0.240550    45.495661   271.431453    238.659824   
2023-12-24   99.410580  0.235473    48.114413   271.254301    233.224955   
2023-12-25   99.848301  0.231077    47.790837   264.652850    229.111577   
2023-12-26  103.189056  0.238222    48.359525   267.178800    234.926906   

                 bitcoin   cardano  chainlink     cosmos       dai  ...  \
timestamp                                                           ...   
2023-12-22  43849.699599  0.636310  15.279277  11.398848  0.999468  ...   
2023-12-23  44003.696022  0.623618  15.497920  11.391916  0.998671  ...   
2023-12-24  43752.030885  0.614803  15.681732  11.314822  0.998909  ...   
2023-12-25  43034.971063  0.593090  15.291259  11.245503  0.998111  ...   
2023-12-26  43638